In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 500)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pickle

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, hp, tpe, space_eval

from sklearn.model_selection import KFold, TimeSeriesSplit
import lightgbm as lgb
from time import time
from tqdm import tqdm_notebook
import qgrid

from xgboost import XGBClassifier
import os

from sklearn.model_selection import KFold
from scipy import stats
from sklearn.metrics import roc_curve

import gc
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
from tqdm import tqdm 
import gc
from time import time
from sklearn.metrics import mean_absolute_error
warnings.filterwarnings('ignore')
import datetime
import matplotlib.pyplot as plt
import os

## 读取数据

In [2]:
df = pd.read_csv('./data/generate_data.csv')

In [3]:
weather = pd.read_csv('./data/天气.csv', sep=',')
del_cols = ['星期']
weather.drop(del_cols, axis=1, inplace=True)

In [4]:
df['日期'] = df['时间'].apply(lambda x: str(x).split(" ")[0])

In [5]:
df = df.merge(weather, on = '日期', how = 'left')

In [6]:
df

,项目,时间,等候时间,日期,最高温,最低温,天气,风力风向,空气质量指数
0,变形金刚,2021-10-19 09:00:00,17.0,2021-10-19,14,1,晴,东北风2级,22
1,变形金刚,2021-10-19 09:15:00,12.0,2021-10-19,14,1,晴,东北风2级,22
2,变形金刚,2021-10-19 09:30:00,42.0,2021-10-19,14,1,晴,东北风2级,22
3,变形金刚,2021-10-19 09:45:00,50.0,2021-10-19,14,1,晴,东北风2级,22
4,变形金刚,2021-10-19 10:00:00,61.0,2021-10-19,14,1,晴,东北风2级,22
...,...,...,...,...,...,...,...,...,...
15480,鹰马飞行,2021-10-01 20:00:00,37.0,2021-10-01,23,16,多云,东北风2级,32
15481,鹰马飞行,2021-10-01 20:15:00,5.0,2021-10-01,23,16,多云,东北风2级,32
15482,鹰马飞行,2021-10-01 20:30:00,10.0,2021-10-01,23,16,多云,东北风2级,32
15483,鹰马飞行,2021-10-01 20:45:00,7.0,2021-10-01,23,16,多云,东北风2级,32


## 特征构造

### 时间特征

In [7]:
def get_time_feature(df, col, keep=False):
    """
    为df增加时间特征列,包括:年,月,日,小时,dayofweek,weekofyear
    :param df:
    :param col: 时间列的列名
    :param keep: 是否保留原始时间列
    :return:
    """
    df_copy = df.copy()
    prefix = col + "_"

    df_copy[col] = pd.to_datetime(df_copy[col])
#     df_copy[prefix + 'year'] = df_copy[col].dt.year
#     df_copy[prefix + 'month'] = df_copy[col].dt.month
    df_copy[prefix + 'day'] = df_copy[col].dt.day
    df_copy[prefix + 'hour'] = df_copy[col].dt.hour
    df_copy[prefix + 'minute'] = df_copy[col].dt.minute
#     df_copy[prefix + 'weekofyear'] = df_copy[col].dt.weekofyear
    df_copy[prefix + 'dayofweek'] = df_copy[col].dt.dayofweek
    df_copy[prefix + 'is_wknd'] = df_copy[col].dt.weekday // 4
    # df_copy[prefix + 'holiday'] = df_copy[[prefix + 'year', prefix + 'month', prefix + 'day']] \
    #                                                 .apply(lambda x: _getholiday(x), axis=1)
    if keep:
        return df_copy
    else:
        return df_copy.drop([col], axis=1)

In [8]:
df = get_time_feature(df, "时间", keep = True)

### lag特征

In [9]:
df = df.sort_values(by = ['时间', '项目'])
df.index = range(len(df))

In [10]:
key = ['项目']
values = ['等候时间']
for value in values:
    stat_temp = df[key + [value]].copy()
    for i in range(1, 9):
        shift_value = stat_temp.groupby(key)[value].shift(i)
        cname = '_'.join(key) + '_' + value + '_lag{}'.format(i)
        df[cname] = shift_value

In [11]:
key = ['项目', '时间_dayofweek']
values = ['等候时间']
for value in values:
    stat_temp = df[key + [value]].copy()
    for i in [1]:
        shift_value = stat_temp.groupby(key)[value].shift(i)
        cname = '_'.join(key) + '_' + value + '_lag{}'.format(i)
        df[cname] = shift_value
        

In [12]:
df

,项目,时间,等候时间,日期,最高温,最低温,天气,风力风向,空气质量指数,时间_day,时间_hour,时间_minute,时间_dayofweek,时间_is_wknd,项目_等候时间_lag1,项目_等候时间_lag2,项目_等候时间_lag3,项目_等候时间_lag4,项目_等候时间_lag5,项目_等候时间_lag6,项目_等候时间_lag7,项目_等候时间_lag8,项目_时间_dayofweek_等候时间_lag1
0,侏罗纪世界,2021-10-01 09:00:00,11.0,2021-10-01,23,16,多云,东北风2级,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,功夫熊猫,2021-10-01 09:00:00,5.0,2021-10-01,23,16,多云,东北风2级,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,变形金刚,2021-10-01 09:00:00,5.0,2021-10-01,23,16,多云,东北风2级,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,哈利波特与禁忌之旅,2021-10-01 09:00:00,5.0,2021-10-01,23,16,多云,东北风2级,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,大黄蜂回旋机,2021-10-01 09:00:00,5.0,2021-10-01,23,16,多云,东北风2级,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15480,灯光,2021-10-19 21:00:00,20.0,2021-10-19,14,1,晴,东北风2级,22,19,21,0,1,0,5.0,28.0,5.0,17.0,14.0,5.0,5.0,5.0,5.0
15481,萌转过山车,2021-10-19 21:00:00,5.0,2021-10-19,14,1,晴,东北风2级,22,19,21,0,1,0,10.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,10.0
15482,超萌漩漩涡,2021-10-19 21:00:00,5.0,2021-10-19,14,1,晴,东北风2级,22,19,21,0,1,0,5.0,5.0,5.0,8.0,18.0,5.0,5.0,5.0,5.0
15483,飞越侏罗纪,2021-10-19 21:00:00,23.0,2021-10-19,14,1,晴,东北风2级,22,19,21,0,1,0,31.0,7.0,16.0,30.0,66.0,22.0,27.0,42.0,31.0


### 统计特征

In [13]:
feas = [x for x in df.columns if '项目_等候时间_lag' in x]
feas

['项目_等候时间_lag1',
 '项目_等候时间_lag2',
 '项目_等候时间_lag3',
 '项目_等候时间_lag4',
 '项目_等候时间_lag5',
 '项目_等候时间_lag6',
 '项目_等候时间_lag7',
 '项目_等候时间_lag8']

In [14]:
temp_feas = feas[:4]
print(temp_feas)

df[f'past_1h_mean'] = df[temp_feas].mean(axis = 1)
df[f'past_1h_median'] = df[temp_feas].median(axis = 1)
df[f'past_1h_std'] = df[temp_feas].std(axis = 1)
df[f'past_1h_min'] = df[temp_feas].min(axis = 1)
df[f'past_1h_max'] = df[temp_feas].max(axis = 1)

['项目_等候时间_lag1', '项目_等候时间_lag2', '项目_等候时间_lag3', '项目_等候时间_lag4']


In [15]:
temp_feas = feas[:8]
print(temp_feas)

df[f'past_2h_mean'] = df[temp_feas].mean(axis = 1)
df[f'past_2h_median'] = df[temp_feas].median(axis = 1)
df[f'past_2h_std'] = df[temp_feas].std(axis = 1)
df[f'past_2h_min'] = df[temp_feas].min(axis = 1)
df[f'past_2h_max'] = df[temp_feas].max(axis = 1)

['项目_等候时间_lag1', '项目_等候时间_lag2', '项目_等候时间_lag3', '项目_等候时间_lag4', '项目_等候时间_lag5', '项目_等候时间_lag6', '项目_等候时间_lag7', '项目_等候时间_lag8']


### 天气信息

### 类别型变量label_encoding

In [16]:
df

,项目,时间,等候时间,日期,最高温,最低温,天气,风力风向,空气质量指数,时间_day,时间_hour,时间_minute,时间_dayofweek,时间_is_wknd,项目_等候时间_lag1,项目_等候时间_lag2,项目_等候时间_lag3,项目_等候时间_lag4,项目_等候时间_lag5,项目_等候时间_lag6,项目_等候时间_lag7,项目_等候时间_lag8,项目_时间_dayofweek_等候时间_lag1,past_1h_mean,past_1h_median,past_1h_std,past_1h_min,past_1h_max,past_2h_mean,past_2h_median,past_2h_std,past_2h_min,past_2h_max
0,侏罗纪世界,2021-10-01 09:00:00,11.0,2021-10-01,23,16,多云,东北风2级,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,功夫熊猫,2021-10-01 09:00:00,5.0,2021-10-01,23,16,多云,东北风2级,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,变形金刚,2021-10-01 09:00:00,5.0,2021-10-01,23,16,多云,东北风2级,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,哈利波特与禁忌之旅,2021-10-01 09:00:00,5.0,2021-10-01,23,16,多云,东北风2级,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,大黄蜂回旋机,2021-10-01 09:00:00,5.0,2021-10-01,23,16,多云,东北风2级,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15480,灯光,2021-10-19 21:00:00,20.0,2021-10-19,14,1,晴,东北风2级,22,19,21,0,1,0,5.0,28.0,5.0,17.0,14.0,5.0,5.0,5.0,5.0,13.75,11.0,11.056672,5.0,28.0,10.500,5.0,8.552360,5.0,28.0
15481,萌转过山车,2021-10-19 21:00:00,5.0,2021-10-19,14,1,晴,东北风2级,22,19,21,0,1,0,10.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,10.0,6.25,5.0,2.500000,5.0,10.0,5.625,5.0,1.767767,5.0,10.0
15482,超萌漩漩涡,2021-10-19 21:00:00,5.0,2021-10-19,14,1,晴,东北风2级,22,19,21,0,1,0,5.0,5.0,5.0,8.0,18.0,5.0,5.0,5.0,5.0,5.75,5.0,1.500000,5.0,8.0,7.000,5.0,4.566962,5.0,18.0
15483,飞越侏罗纪,2021-10-19 21:00:00,23.0,2021-10-19,14,1,晴,东北风2级,22,19,21,0,1,0,31.0,7.0,16.0,30.0,66.0,22.0,27.0,42.0,31.0,21.00,23.0,11.575837,7.0,31.0,30.125,28.5,17.884051,7.0,66.0


In [17]:
from sklearn import preprocessing
from tqdm import tqdm_notebook

for f in tqdm_notebook(df.columns):
    if f in ['时间','等候时间','日期', '项目']:
        continue
    if 'O' == df[f].dtype:
        lbl = LabelEncoder()
        print(f, df[f].nunique())
        lbl.fit(df[f])
        df[f] = lbl.transform(list(df[f].astype(str)))

天气 4
风力风向 10



## 切分数据集

In [18]:
df['时间'].max(), df['时间'].min()

(Timestamp('2021-10-19 21:00:00'), Timestamp('2021-10-01 09:00:00'))

In [19]:
train = df.loc[~(df['时间'] > '2021-10-18 21:00:00')]
test = df.loc[df['时间'] > '2021-10-18 21:00:00']

In [20]:
train['时间'].min(), train['时间'].max(), 

(Timestamp('2021-10-01 09:00:00'), Timestamp('2021-10-18 21:00:00'))

In [21]:
test['时间'].min(), test['时间'].max()

(Timestamp('2021-10-19 09:00:00'), Timestamp('2021-10-19 21:00:00'))

In [22]:
target = '等候时间'

### target_encoder

In [23]:
cols = ['项目', '时间_day', '时间_hour', '时间_minute', '时间_dayofweek',
       '时间_is_wknd']
for col in cols:
    temp_map = pd.DataFrame(train.groupby([col])[target].mean()).reset_index()
    temp_map.rename({target: col + '_target_mean'}, axis=1, inplace=True)
    train = pd.merge(train, temp_map, on = col, how = 'left')
    test  = pd.merge(test,  temp_map, on = col, how = 'left')

## 切分验证集

In [24]:
train['时间'].min(), train['时间'].max(), 

(Timestamp('2021-10-01 09:00:00'), Timestamp('2021-10-18 21:00:00'))

In [25]:
split_time = '2021-10-18 08:00:00'

# 切线下验证集: 最后一部分的数据作为线下验证
valid_idx = train.loc[train['时间'] >= split_time].index
train_idx = train.loc[~(train['时间'] >= split_time)].index

In [26]:
assert(len(valid_idx) + len(train_idx) == train.shape[0])

## 模型

In [27]:
train.head()

,项目,时间,等候时间,日期,最高温,最低温,天气,风力风向,空气质量指数,时间_day,时间_hour,时间_minute,时间_dayofweek,时间_is_wknd,项目_等候时间_lag1,项目_等候时间_lag2,项目_等候时间_lag3,项目_等候时间_lag4,项目_等候时间_lag5,项目_等候时间_lag6,项目_等候时间_lag7,项目_等候时间_lag8,项目_时间_dayofweek_等候时间_lag1,past_1h_mean,past_1h_median,past_1h_std,past_1h_min,past_1h_max,past_2h_mean,past_2h_median,past_2h_std,past_2h_min,past_2h_max,项目_target_mean,时间_day_target_mean,时间_hour_target_mean,时间_minute_target_mean,时间_dayofweek_target_mean,时间_is_wknd_target_mean
0,侏罗纪世界,2021-10-01 09:00:00,11.0,2021-10-01,23,16,1,1,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.028935,22.899387,28.140523,32.420521,22.877301,34.335924
1,功夫熊猫,2021-10-01 09:00:00,5.0,2021-10-01,23,16,1,1,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.357143,22.899387,28.140523,32.420521,22.877301,34.335924
2,变形金刚,2021-10-01 09:00:00,5.0,2021-10-01,23,16,1,1,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.673611,22.899387,28.140523,32.420521,22.877301,34.335924
3,哈利波特与禁忌之旅,2021-10-01 09:00:00,5.0,2021-10-01,23,16,1,1,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.182870,22.899387,28.140523,32.420521,22.877301,34.335924
4,大黄蜂回旋机,2021-10-01 09:00:00,5.0,2021-10-01,23,16,1,1,32,1,9,0,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.460648,22.899387,28.140523,32.420521,22.877301,34.335924


In [28]:
target = '等候时间'
not_used = [target] + ['时间', '日期', '项目']
used_features = [x for x in test.columns if x not in not_used]
print(train[used_features].shape, test[used_features].shape, train[target].shape)

(14670, 35) (815, 35) (14670,)


In [29]:
# target取log
target_log = True
if target_log:    
    train[target] = train[target].apply(lambda x: np.log1p(x))

In [30]:
lr = 0.006883242363721497
Early_Stopping_Rounds = 300

N_round = 5000  # 5000
Verbose_eval = 100

params =  {'num_leaves': 61,  # 当前base 61
           'min_child_weight': 0.03454472573214212,
           'feature_fraction': 0.3797454081646243,
           'bagging_fraction': 0.4181193142567742,
           'min_data_in_leaf': 96,  # 当前base 106
           'objective': 'regression',
           "metric": 'mae',
           'max_depth': -1,
           'learning_rate': lr,
           "boosting_type": "gbdt",
           "bagging_seed": 11,
           "verbosity": -1,
           'reg_alpha': 0.3899927210061127,
           'reg_lambda': 0.6485237330340494,
           'random_state': 47,
           'num_threads': 16,
           'lambda_l1': 1,  
           'lambda_l2': 1,
           'verbose': -1
    #      'is_unbalance':True
            
             }

In [31]:
sub = test[['项目', '时间']]
lgb_sub = sub
lgb_sub[target] = 0

MAEs = []
feature_importances = pd.DataFrame()
feature_importances['feature'] = train[used_features].columns


N_MODEL = 1.0 
for model_i in tqdm(range(int(N_MODEL))):

    if N_MODEL != 1.0:
        params['seed'] = model_i + 1123

    start_time = time()
    print('Training on model {}'.format(model_i + 1))
    
    ## All data
    trn_data = lgb.Dataset(train.iloc[train_idx][used_features], label=train.iloc[train_idx][target],
                           categorical_feature='')
    val_data = lgb.Dataset(train.iloc[valid_idx][used_features], label=train.iloc[valid_idx][target],
                           categorical_feature='')
    
    clf = lgb.train(params, trn_data, num_boost_round=N_round, valid_sets=[trn_data, val_data],
                    verbose_eval=Verbose_eval, early_stopping_rounds=Early_Stopping_Rounds)  # , feval=evalerror
    
    val = clf.predict(train.iloc[valid_idx][used_features])
    
    if target_log:
        mae_ = mean_absolute_error(np.expm1(train.iloc[valid_idx][target]), np.expm1(val))
    else:
        mae_ = mean_absolute_error(train.iloc[valid_idx][target], val)
    
    print('MAE: {}'.format(mae_))
    
    MAEs.append(mae_)
    feature_importances['model_{}'.format(model_i + 1)] = clf.feature_importance()
    
    ## 用数据集重新训练
    print("ReTraining on all data")
    
    gc.enable()
    del trn_data, val_data
    gc.collect()
    Best_iteration = clf.best_iteration
    print("Best_iteration: ", Best_iteration)
    trn_data = lgb.Dataset(train[used_features], label=train[target], categorical_feature='')
    clf = lgb.train(params, trn_data, num_boost_round=int(Best_iteration * 1.02),
                    valid_sets=[trn_data], verbose_eval=Verbose_eval)  # , feval=evalerror
    
    pred = clf.predict(test[used_features])
    lgb_sub[target] = lgb_sub[target] + pred / N_MODEL
    
    print('Model {} finished in {}'.format(model_i + 1, str(datetime.timedelta(seconds=time() - start_time))))

  0%|          | 0/1 [00:00<?, ?it/s]

Training on model 1
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
[LightGBM] [Warning] lambda_l1 is set=1, reg_alpha=0.3899927210061127 will be ignored. Current value: lambda_l1=1
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.6485237330340494 will be ignored. Current value: lambda_l2=1
Training until validation scores don't improve for 300 rounds
[100]	training's l1: 0.607837	valid_1's l1: 0.611895
[200]	training's l1: 0.53987	valid_1's l1: 0.542698
[300]	training's l1: 0.507663	valid_1's l1: 0.51602
[400]	training's l1: 0.489525	valid_1's l1: 0.505253
[500]	training's l1: 0.4777	valid_1's l1: 0.500655
[600]	training's l1: 0.468048	valid_1's l1: 0.497392
[700]	training's l1: 0.459973	valid_1's l1: 0.495856
[800]	training's l1: 0.452707	valid_1's l1: 0.495206
[900]	training's l1: 0.446166	valid_1's l1: 0.495148
[1000]	training's l1: 0.439972	valid_1's l1: 0.495024
[1100]	training's l1: 0.434125	valid_1's l1: 0.495053
[1200]	tra

100%|██████████| 1/1 [00:09<00:00,  9.21s/it]

Model 1 finished in 0:00:09.206660


In [32]:
if target_log:    
    lgb_sub[target] = lgb_sub[target].apply(lambda x: np.expm1(x))

In [33]:
feature_importances['average'] = feature_importances[[x for x in feature_importances.columns if x != "feature"]].mean(axis=1)
feature_importances = feature_importances.sort_values(by = "average", ascending = False)
# feature_importances.to_csv('feature_importances.csv')
feature_importances


,feature,model_1,average
29,项目_target_mean,4792,4792.0
21,past_1h_std,4266,4266.0
26,past_2h_std,4124,4124.0
24,past_2h_mean,3622,3622.0
6,时间_hour,3480,3480.0
16,项目_等候时间_lag7,3409,3409.0
19,past_1h_mean,3319,3319.0
17,项目_等候时间_lag8,3227,3227.0
13,项目_等候时间_lag4,3158,3158.0
14,项目_等候时间_lag5,3015,3015.0


In [34]:
mean_absolute_error(test[target], lgb_sub[target])

11.63082928703361

In [35]:
# import matplotlib.pyplot as plt

# plt.plot(range(lgb_sub.shape[0]),test[target],"x-",label="migration time")
# plt.plot(range(lgb_sub.shape[0]),lgb_sub[target],"+-",label="request delay")
 
# """open the grid"""
# plt.grid(True)

# plt.legend(bbox_to_anchor=(1.0, 1), loc=1, borderaxespad=0.)

# plt.show()

In [36]:
used_features

['最高温',
 '最低温',
 '天气',
 '风力风向',
 '空气质量指数',
 '时间_day',
 '时间_hour',
 '时间_minute',
 '时间_dayofweek',
 '时间_is_wknd',
 '项目_等候时间_lag1',
 '项目_等候时间_lag2',
 '项目_等候时间_lag3',
 '项目_等候时间_lag4',
 '项目_等候时间_lag5',
 '项目_等候时间_lag6',
 '项目_等候时间_lag7',
 '项目_等候时间_lag8',
 '项目_时间_dayofweek_等候时间_lag1',
 'past_1h_mean',
 'past_1h_median',
 'past_1h_std',
 'past_1h_min',
 'past_1h_max',
 'past_2h_mean',
 'past_2h_median',
 'past_2h_std',
 'past_2h_min',
 'past_2h_max',
 '项目_target_mean',
 '时间_day_target_mean',
 '时间_hour_target_mean',
 '时间_minute_target_mean',
 '时间_dayofweek_target_mean',
 '时间_is_wknd_target_mean']

In [37]:
show = test[['项目', '时间', '等候时间']]
show['pre'] = lgb_sub['等候时间']

In [38]:
show

,项目,时间,等候时间,pre
0,侏罗纪世界,2021-10-19 09:00:00,5.0,15.579827
1,功夫熊猫,2021-10-19 09:00:00,18.0,9.804105
2,变形金刚,2021-10-19 09:00:00,17.0,22.527546
3,哈利波特与禁忌之旅,2021-10-19 09:00:00,43.0,33.237443
4,大黄蜂回旋机,2021-10-19 09:00:00,13.0,12.886407
...,...,...,...,...
810,灯光,2021-10-19 21:00:00,20.0,8.550728
811,萌转过山车,2021-10-19 21:00:00,5.0,11.847240
812,超萌漩漩涡,2021-10-19 21:00:00,5.0,9.728566
813,飞越侏罗纪,2021-10-19 21:00:00,23.0,12.792938


In [39]:
test.to_csv('test.csv', index = False)

In [40]:
# dump model with pickle
with open('model.pkl', 'wb') as fout:
    pickle.dump(clf, fout)

## 使用

In [42]:
used_features = ['最高温',
 '最低温',
 '天气',
 '风力风向',
 '空气质量指数',
 '时间_day',
 '时间_hour',
 '时间_minute',
 '时间_dayofweek',
 '时间_is_wknd',
 '项目_等候时间_lag1',
 '项目_等候时间_lag2',
 '项目_等候时间_lag3',
 '项目_等候时间_lag4',
 '项目_等候时间_lag5',
 '项目_等候时间_lag6',
 '项目_等候时间_lag7',
 '项目_等候时间_lag8',
 '项目_时间_dayofweek_等候时间_lag1',
 'past_1h_mean',
 'past_1h_median',
 'past_1h_std',
 'past_1h_min',
 'past_1h_max',
 'past_2h_mean',
 'past_2h_median',
 'past_2h_std',
 'past_2h_min',
 'past_2h_max',
 '项目_target_mean',
 '时间_day_target_mean',
 '时间_hour_target_mean',
 '时间_minute_target_mean',
 '时间_dayofweek_target_mean',
 '时间_is_wknd_target_mean']

In [43]:
target = '等候时间'

In [44]:
import pandas as pd
import pickle

In [45]:
test = pd.read_csv("test.csv")

In [46]:
# load model with pickle to predict
with open('model.pkl', 'rb') as fin:
    pkl_bst = pickle.load(fin)

In [47]:
import numpy as np
def clip_label(pred, min_, max_):
    pred = np.clip(pred, min_, max_)
    return pred

In [48]:
import time
import time
from datetime import datetime, timedelta
cur_date = "2021-10-19 09:00:00"
for _ in range(10):
    for i in range(40):
        delta_time = timedelta(minutes=15 * i)
        cur_time = str(datetime.strptime(cur_date, '%Y-%m-%d %H:%M:%S') + delta_time)
        print(cur_time)

        cur = test.loc[test['时间'] == cur_time, :]
        cur_res = cur[['项目', '时间']]
        y_pred = pkl_bst.predict(cur[used_features])
        y_pred = np.expm1(y_pred)
        y_pred = clip_label(y_pred, 5, 120)
        cur_res[target] = y_pred
        cur_res[target] = cur_res[target].apply(lambda x: round(x, 0))
        for item in cur_res['项目'].unique():
            cur_res.loc[cur_res['项目'] == item, '等候时间'].to_csv(f'./sub/{item}.txt', index = False, header = False)
        cur_res.index = range(len(cur_res))
        cur_res.loc[0,['时间']].to_csv(f'./sub/time.txt', index = False, header = False)
        time.sleep(3)

2021-10-19 09:00:00
2021-10-19 09:15:00
2021-10-19 09:30:00
2021-10-19 09:45:00
2021-10-19 10:00:00
2021-10-19 10:15:00
2021-10-19 10:30:00
2021-10-19 10:45:00
2021-10-19 11:00:00
2021-10-19 11:15:00
2021-10-19 11:30:00
2021-10-19 11:45:00
2021-10-19 12:00:00
2021-10-19 12:15:00
2021-10-19 12:30:00
2021-10-19 12:45:00
2021-10-19 13:00:00
2021-10-19 13:15:00
2021-10-19 13:30:00
2021-10-19 13:45:00
2021-10-19 14:00:00
2021-10-19 14:15:00
2021-10-19 14:30:00
2021-10-19 14:45:00
2021-10-19 15:00:00
2021-10-19 15:15:00
2021-10-19 15:30:00
2021-10-19 15:45:00
2021-10-19 16:00:00
2021-10-19 16:15:00
2021-10-19 16:30:00
2021-10-19 16:45:00
2021-10-19 17:00:00
2021-10-19 17:15:00
2021-10-19 17:30:00
2021-10-19 17:45:00
2021-10-19 18:00:00
2021-10-19 18:15:00
2021-10-19 18:30:00
2021-10-19 18:45:00
2021-10-19 09:00:00
2021-10-19 09:15:00
2021-10-19 09:30:00
2021-10-19 09:45:00
2021-10-19 10:00:00
2021-10-19 10:15:00
2021-10-19 10:30:00
2021-10-19 10:45:00
2021-10-19 11:00:00
2021-10-19 11:15:00


KeyboardInterrupt: 